In [1]:
import pyspark
import os
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

conf = (SparkConf()
       .set("spark.driver.bindAddress", "0.0.0.0")\
       .set("spark.submit.deployMode", "client")\
       .set("spark.kubernetes.driver.pod.name", os.getenv("MY_POD_NAME"))\
       .set("spark.kubernetes.authenticate.subdmission.caCertFile", "/var/run/secrets/kubernetes.io/serviceaccount/ca.crt")\
       .set("spark.kubernetes.authenticate.submission.oauthTokenFile", "/var/run/secrets/kubernetes.io/serviceaccount/token")\
       .set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark-driver")\
       .set("spark.kubernetes.namespace", os.getenv("MY_POD_NAMESPACE"))\
       .set("spark.executor.instances", "1")\
       .set("spark.dynamicAllocation.maxExecutors", "2")\
       .set("spark.dynamicAllocation.enabled", "true")\
       .set("spark.dynamicAllocation.shuffleTracking.enabled", "true")\
       .set("spark.driver.host", "spark-driver")\
       .set("spark.driver.port", "20020")\
       .set("spark.kubernetes.executor.request.cores", "1")\
       .set("spark.kubernetes.executor.limit.cores", "2")\
       .set("spark.executor.memory", "8000m")\
       .set("spark.kubernetes.container.image", "klovercloud/airflow-spark-k8s-driver:3.1.2")\
       .set("spark.kubernetes.container.image.pullPolicy", "IfNotPresent")\
       .set("spark.sql.extensions","org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")\
       .set("spark.sql.catalog.spark_catalog","org.apache.iceberg.spark.SparkSessionCatalog")\
       .set("spark.sql.catalog.spark_catalog.type","hive")\
       .set("spark.hadoop.hive.metastore.uris","thrift://hive-metastore.analytics-dev:9083")\
       .set("spark.hadoop.hive.metastore.warehouse.dir","s3a://bigdata-dev-cmfcknil/warehouse/")\
       .set("spark.hadoop.fs.s3a.connection.ssl.enabled","true")\
       .set("spark.hadoop.fs.s3a.path.style.access","true")\
       .set("spark.hadoop.fs.s3a.endpoint","https://kcs3.bd-1.wpc.waltonelectronics.com")\
       .set("spark.hadoop.fs.s3a.access.key","30J9IFFY75NVQZVHG4D4")\
       .set("spark.hadoop.fs.s3a.secret.key","WMGk6kqq5erjqZjnqo0QItSQ2zuoNWEpHIphsTgR")\
       .set("spark.hadoop.fs.s3a.attempts.maximum","1")\
       .set("spark.hadoop.fs.s3a.connection.establish.timeout","500")\
       .set("spark.hadoop.datanucleus.autoCreateSchema","true")\
       .set("spark.hadoop.datanucleus.fixedDatastore","false"));

spark = SparkSession.builder \
            .master("k8s://https://kubernetes.default:443") \
            .appName("spark-test-app") \
            .config(conf=conf).enableHiveSupport().getOrCreate()

22/02/18 16:43:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/18 16:43:19 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


In [2]:
spark.sql("SHOW DATABASES;").show()

+---------+
|namespace|
+---------+
|  default|
|      ebs|
|      pos|
|   pqspos|
|      reg|
|     test|
|    tmpdb|
+---------+



In [3]:
dfM = spark.sql("select INVOICE_NO,CREATED,CREATED_BY,S_TYPE,IS_EMPLOYEE,CUSTOMER_ID,ID,PLAZA_ID from pos.pos___POS_SALE_MST;")   # Sales Invoice Master
dfD = spark.sql("select RATE,QUANTITY,S_TOTAL,REBATE,G_TOTAL,SALE_MST_ID,PRODUCT_ID from pos.pos___POS_SALE_DTLS;")
dfPD = spark.sql("select SEGMENT3,SEGMENT5,ID from pos.pos___POS_PRODUCTS;")
dfC = spark.sql("select NAME,MOBILE_PHONE,C_TYPE,ID from pos.pos___POS_CUSTOMERS;")
dfP = spark.sql("select NAME,ID from pos.pos___POS_PLAZAS;")

22/02/18 16:43:43 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [4]:
dfP = dfP.withColumnRenamed("ID","pid")


In [5]:
dfM = dfM.withColumnRenamed("ID","SID")


In [6]:
dfC = dfC.withColumnRenamed("ID","CID")
dfC = dfC.withColumnRenamed("NAME","cname")
dfC = dfC.withColumnRenamed("MOBILE_PHONE","CUSTOMER_PHONE_NO")
dfC = dfC.withColumnRenamed("C_TYPE","CUSTOMER_TYPE")


In [7]:
dfPD = dfPD.withColumnRenamed("ID","prodid")


In [8]:
dfD= dfD.filter(dfD.RATE > 1500)

In [9]:
df1 = dfM.join(dfD,dfM.SID == dfD.SALE_MST_ID,"inner")

In [10]:
df2= df1.join(dfPD, dfPD.prodid== df1.PRODUCT_ID,"inner")

In [11]:
df3 = dfC.join(df2,dfC.CID == df2.CUSTOMER_ID,"inner")

In [12]:
df4 = dfP.join(df3,dfP.pid == df3.PLAZA_ID,"inner")

In [13]:
df4 = df4.withColumnRenamed("Name","PLAZA_NAME")

In [14]:
df4 = df4.drop("pid")

In [15]:
df4 = df4.withColumnRenamed("cname","CustomerName")

In [16]:
df4 = df4.drop("CID")

In [17]:
df4 = df4.filter(df4.IS_EMPLOYEE == False)

In [18]:
#df4 = df4.where('''Customer_Type in ('Retail','Hire')''')
df4 = df4.filter(df4.CUSTOMER_TYPE.contains('Retail')|df4.CUSTOMER_TYPE.contains('Hire'))

In [19]:
df4 = df4.filter(df4.CREATED_BY != "OPENING_HIRE")

In [20]:
import pyspark.sql.functions as funcs
df4 = df4.withColumn("date_of_registration",funcs.to_date(funcs.col("CREATED")))

In [21]:
df4 = df4.drop("CREATED")

In [22]:
from pyspark.sql.functions import *
from pyspark.sql.functions import when, lit, col
import pyspark.sql.functions as F
 
 
 
df4 = df4.filter(df4["date_of_registration"] >= lit('2021-01-01')) \
                   .filter(df4["date_of_registration"] <= lit('2022-02-18'))



In [23]:
df = df4.cache()

In [24]:
df = df.withColumnRenamed("CUSTOMER_PHONE_NO","CustomerPhoneNumber")

## Preprocessing CustomerPhoneNumber

In [25]:
df = df.filter(df.CustomerPhoneNumber.isNotNull())

## Remove zeros from customerphonenumber column

In [26]:
df = df.filter(df.CustomerPhoneNumber != "0")

## Remove leading plus minus sign from customerphonenumber column

In [27]:

#Remove leading plus minus sign
from pyspark.sql.functions import *

import pyspark.sql.functions as F

 

df = df.withColumn('CustomerPhoneNumber', F.regexp_replace('CustomerPhoneNumber', r'^[±]*', ''))

## Remove punctuation from customerphonenumber column

In [28]:
### Remove punctuation from column in pyspark
df = df.withColumn('CustomerPhoneNumber', F.regexp_replace('CustomerPhoneNumber', r'[^\w\s]', ''))

## Remove all special characters from CustomerPhoneNumber column

In [29]:
df = df.withColumn('CustomerPhoneNumber', F.regexp_replace('CustomerPhoneNumber', r'\W', ''))

## Remove leading eight from CustomerPhoneNumber column

In [30]:
df = df.withColumn('CustomerPhoneNumber', F.regexp_replace('CustomerPhoneNumber', r'^[8]*', ''))

In [31]:
## Dropping na values from customerphonenumber column

In [32]:
df = df.na.drop(subset=["CustomerPhoneNumber"])

## Remove english alphabets from customerphonenumber column

In [33]:
df = df.withColumn('CustomerPhoneNumber', F.regexp_replace('CustomerPhoneNumber', r'[A-Za-z]', ''))

## Removing empty strings

In [34]:
df = df.filter(df.CustomerPhoneNumber != "")

## Remove leading plus sign from customerphonenumber column

In [35]:
#remove leading plus
df = df.withColumn('CustomerPhoneNumber', F.regexp_replace('CustomerPhoneNumber', r'^[+]*', ''))

## Remove | from customerphonenumber column

In [36]:

df = df.withColumn('CustomerPhoneNumber', F.regexp_replace('CustomerPhoneNumber', r'|', ''))

## Remove / from customerphonenumber column

In [37]:
df = df.withColumn('CustomerPhoneNumber', F.regexp_replace('CustomerPhoneNumber', r'/', ''))

## Remove all non-english characters

In [38]:
df = df.withColumn('CustomerPhoneNumber', F.regexp_replace('CustomerPhoneNumber', r'/[^\x00-\x7F]+/', ''))

## Removing empty strings after final data cleaning

In [39]:
df = df.filter(df.CustomerPhoneNumber != "")

## Removing all spaces from CustomerPhonenumber column

In [40]:
df = df.withColumn('CustomerPhoneNumber', F.regexp_replace('CustomerPhoneNumber', r'/\s/g', ''))

## Finding the length of customerphonenumber in customerphonenumber column

In [41]:
from pyspark.sql.functions import length
df = df.withColumn("length", F.length("CustomerPhoneNumber"))

## Remove all customers who do not have 11 digit number

In [42]:
df = df.filter(df.length == 11)

## Select all customers whose phone number starts with 01

In [43]:
df = df.filter(df.CustomerPhoneNumber.startswith('01'))

## Remove all customers whose phone numbers start with 010

In [44]:
df = df.filter(~df.CustomerPhoneNumber.startswith('010'))

## Remove all customers whose phone numbers start with 011

In [45]:
df = df.filter(~df.CustomerPhoneNumber.startswith('011'))

## Remove all customers whose phone numbers start with 012

In [46]:
df = df.filter(~df.CustomerPhoneNumber.startswith('012'))

In [47]:
df = df.withColumn('remaining_strings', substring('CustomerPhoneNumber', 4, 11))

In [48]:
df = df.filter(df.remaining_strings != "00000000")

In [49]:
df = df.withColumn('remaining_new', substring('remaining_strings', 1, 7))

In [50]:
df = df.filter(df.remaining_new != "0000000")

In [51]:
from pyspark.sql.functions import when

df = df.withColumn('SEGMENT3',
    F.when((df.SEGMENT3 == "LAPTOP") | (df.SEGMENT3 == 'DESKTOP PC') | (df.SEGMENT3 == 'SERVER') | (df.SEGMENT3 == 'ALL IN ONE'), "COMPUTER").otherwise(df.SEGMENT3))


In [52]:
df = df.withColumn('SEGMENT3',
    F.when((df.SEGMENT3 == "3D TELEVISION") , "LED SMART TELEVISION").otherwise(df.SEGMENT3))

In [53]:
df = df.withColumn('SEGMENT3',
    F.when((df.SEGMENT3 == "LCD TELEVISION") , "LED TELEVISION").otherwise(df.SEGMENT3))

In [54]:
df = df.withColumn('SEGMENT3',
    F.when((df.SEGMENT3 == "BEVERAGE COOLER") , "REFRIGERATOR").otherwise(df.SEGMENT3))

In [55]:
df = df.withColumn('SEGMENT3',
    F.when((df.SEGMENT3 == "AIRCONDITIONER-INDOOR") , "AIRCONDITIONER").otherwise(df.SEGMENT3))

In [56]:
dfselected = df.select("CustomerPhoneNumber","SEGMENT3","date_of_registration")

In [57]:
dfcountfinal = df.groupby('CustomerPhoneNumber','SEGMENT3').agg(funcs.expr('count(SEGMENT3)')\
                              .alias('productcount'))

In [58]:
dfcountfinal = dfcountfinal.filter(dfcountfinal.productcount < 5)

In [59]:
dfcountfinalaircon = dfcountfinal.filter(dfcountfinal.SEGMENT3 == "AIRCONDITIONER")

In [60]:
dfcountfinalaircon = dfcountfinalaircon.filter(dfcountfinalaircon.productcount <= 4)

In [61]:
dfcountfinalcomputer = dfcountfinal.filter(dfcountfinal.SEGMENT3 == "COMPUTER")

In [62]:
dfcountfinalcomputer = dfcountfinalcomputer.filter(dfcountfinalcomputer.productcount <= 3)

In [63]:
dfcountfinalfreezer = dfcountfinal.filter(dfcountfinal.SEGMENT3 == "FREEZER")

In [64]:
dfcountfinalfreezer = dfcountfinalfreezer.filter(dfcountfinalfreezer.productcount <= 3)

In [65]:
dfcountfinaltelevision = dfcountfinal.filter((dfcountfinal.SEGMENT3 == "LED TELEVISION") | (dfcountfinal.SEGMENT3 == "LED SMART TELEVISION"))

In [66]:
dfcountfinaltelevision = dfcountfinaltelevision.filter(dfcountfinaltelevision.productcount <= 4)

In [67]:
dfcountfinalmicrowaveoven = dfcountfinal.filter(dfcountfinal.SEGMENT3 == "MICROWAVE OVEN")

In [68]:
dfcountfinalmicrowaveoven = dfcountfinalmicrowaveoven.filter(dfcountfinalmicrowaveoven.productcount <= 4)

In [69]:
dfcountfinalrefrigerator = dfcountfinal.filter(dfcountfinal.SEGMENT3 == "REFRIGERATOR")

In [70]:
dfcountfinalrefrigerator = dfcountfinalrefrigerator.filter(dfcountfinalrefrigerator.productcount <= 4)

In [71]:
dfcountfinalwashingmachine = dfcountfinal.filter(dfcountfinal.SEGMENT3 == "WASHING MACHINE")

In [72]:
dfcountfinalwashingmachine = dfcountfinalwashingmachine.filter(dfcountfinalwashingmachine.productcount <= 3)

In [73]:
resultant = dfcountfinalaircon.union(dfcountfinalcomputer)

In [74]:
resultant1 = resultant.union(dfcountfinalfreezer)

In [75]:
resultant2 = resultant1.union(dfcountfinaltelevision)

In [76]:
resultant3 = resultant2.union(dfcountfinalmicrowaveoven)

In [77]:
resultant4 = resultant3.union(dfcountfinalrefrigerator)

In [78]:
resultant5 = resultant4.union(dfcountfinalwashingmachine)

In [79]:
resultant5 = resultant5.withColumnRenamed("CustomerPhoneNumber","Number")

In [80]:
dfselected = dfselected.withColumnRenamed("SEGMENT3","Product")

In [81]:
resultant5.registerTempTable("resultant5")
dfselected.registerTempTable("dfselected")

/usr/local/lib/python3.8/dist-packages/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(
22/02/18 16:44:20 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [82]:
new = spark.sql("SELECT dfselected.CustomerPhoneNumber,resultant5.SEGMENT3,dfselected.date_of_registration,resultant5.productcount from resultant5 inner join dfselected on resultant5.Number = dfselected.CustomerPhoneNumber and resultant5.SEGMENT3 = dfselected.Product")

In [83]:
df = new.select("CustomerPhoneNumber")

In [84]:
df = df.withColumn('lf5', substring('CustomerPhoneNumber', 7, 1))

In [85]:
from pyspark.sql import functions as funcs
 
df = df.groupby('lf5').agg(funcs.expr('count(lf5)')\
                              .alias('Frequency'))

In [86]:
df = df.dropDuplicates()

In [87]:
df.show(2)

22/02/18 16:44:47 ERROR CodeGenerator: failed to compile: org.codehaus.janino.InternalCompilerException: Compiling "GeneratedClass" in "generated.java": Code of method "processNext()V" of class "org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage16" grows beyond 64 KB
org.codehaus.janino.InternalCompilerException: Compiling "GeneratedClass" in "generated.java": Code of method "processNext()V" of class "org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage16" grows beyond 64 KB
	at org.codehaus.janino.UnitCompiler.compile2(UnitCompiler.java:366)
	at org.codehaus.janino.UnitCompiler.access$000(UnitCompiler.java:226)
	at org.codehaus.janino.UnitCompiler$1.visitCompilationUnit(UnitCompiler.java:336)
	at org.codehaus.janino.UnitCompiler$1.visitCompilationUnit(UnitCompiler.java:333)
	at org.codehaus.janino.Java$CompilationUnit.accept(Java.java:363)
	at org.codehaus.janino.UnitCompiler.compileUnit(UnitCompiler.java:333)

+---+---------+
|lf5|Frequency|
+---+---------+
|  7|    65943|
|  3|    67104|
+---+---------+
only showing top 2 rows



In [88]:
df.repartition(1).write.format("csv").options(header='True',delimiter='|',encoding ='UTF-16').save("s3a://bigdata-dev-cmfcknil/raw/init/posdb/pos/lf56")

22/02/18 17:11:53 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/02/18 17:11:55 ERROR CodeGenerator: failed to compile: org.codehaus.janino.InternalCompilerException: Compiling "GeneratedClass" in "generated.java": Code of method "processNext()V" of class "org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage16" grows beyond 64 KB
org.codehaus.janino.InternalCompilerException: Compiling "GeneratedClass" in "generated.java": Code of method "processNext()V" of class "org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage16" grows beyond 64 KB
	at org.codehaus.janino.UnitCompiler.compile2(UnitCompiler.java:366)
	at org.codehaus.janino.UnitCompiler.access$000(UnitCompiler.java:226)
	at org.codehaus.janino.UnitCompiler$1.visitCompilationUnit(UnitCompiler.java:336)
	at org.codehaus.janino.UnitCompiler$1.visitCompilationUnit(UnitCompiler.java:333